In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ["OPENAI_API_KEY"] 

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken


In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [4]:
txt_file_path = "/home/lillian/Documents/TenAcademy/week11/Contract-AI-Chatbot/backend/data/contract.txt"

In [5]:
loader = TextLoader(file_path= txt_file_path, encoding="utf-8")
data = loader.load()

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=5100, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [7]:
embeddings = OpenAIEmbeddings()

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [9]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4",max_tokens=500)

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)

In [10]:
from langchain_core.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT_2 = PromptTemplate.from_template(_template)

prompt_template = """ You are a legal expert who knows and understands legal documents, you should give answer to the based on the contexts you will be given.

{context}

Question: {question}
Helpful Answer:"""
QA_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
print (type(CONDENSE_QUESTION_PROMPT_2))
print (type(QA_PROMPT))


<class 'langchain_core.prompts.prompt.PromptTemplate'>
<class 'langchain_core.prompts.prompt.PromptTemplate'>


In [42]:
prompt = "You are a legal expert who knows and understands legal documents"
CONDENSE_QUESTION_PROMPT_2 = PromptTemplate.from_template(prompt)

In [46]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory,
    
)
# conversation = ConversationChain(
#     llm=llm,
#     verbose=True,
#     prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
#     memory=ConversationEntityMemory(llm=llm,),
# )

In [14]:
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
# combine_docs_chain = StuffDocumentsChain()
retriever = vectorstore.as_retriever()
# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.

from langchain.chains import StuffDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Summarize this content: {context}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
template = (
    "Only when necessary: Combine the chat history and follow up question into "
    "a standalone question to give accurate answer. Chat History: {chat_history}"
    "Give yes or no answer {question}"
)
prompt = PromptTemplate.from_template(template)
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    combine_docs_chain=chain,
    retriever=retriever,
    question_generator=question_generator_chain,
    memory=memory
)

In [17]:
query = "How much is the escrow amount?"
result = chain({"question": query})
answer = result["answer"]
answer

"The document describes an agreement between parties, which includes provisions related to the decision of an Accounting Firm, purchase price adjustment, escrow, withholding rights, payments of employee closing bonus, reliance on appointment, and the entirety of the agreement. \n\nThe agreement entails that if the final closing cash amount, closing debt amount, or seller transaction expenses differ from the estimated amounts, the purchase price will be recalculated. The agreement also includes the process for disbursement from the escrow account in case of a dispute.\n\nThe agreement provides for withholding rights, indicating that each buyer or anyone acting on its behalf, has the right to deduct and withhold from the closing cash payment and any other payments required under the agreement. \n\nThe agreement further outlines the process for payments of an employee closing bonus, specifying that the buyer will deposit the employee closing bonus amount with the company and will arrange 

In [18]:
from IPython.display import display, Markdown

formatted_answer = f"**Answer:** \n {answer}"
display(Markdown(formatted_answer))

**Answer:** 
 The document describes an agreement between parties, which includes provisions related to the decision of an Accounting Firm, purchase price adjustment, escrow, withholding rights, payments of employee closing bonus, reliance on appointment, and the entirety of the agreement. 

The agreement entails that if the final closing cash amount, closing debt amount, or seller transaction expenses differ from the estimated amounts, the purchase price will be recalculated. The agreement also includes the process for disbursement from the escrow account in case of a dispute.

The agreement provides for withholding rights, indicating that each buyer or anyone acting on its behalf, has the right to deduct and withhold from the closing cash payment and any other payments required under the agreement. 

The agreement further outlines the process for payments of an employee closing bonus, specifying that the buyer will deposit the employee closing bonus amount with the company and will arrange for the company to pay each employee their respective share.

The agreement also covers aspects such as the appointment and authority of the Sellers' Representative, the entire agreement, facsimile signature, and the severability of the agreement.

In [17]:
questions = [

    "Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?",
    
    "Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?",
    "How much is the escrow amount?",
    "Is the escrow amount greater than the Retention Amount?",
    "What is the purpose of the escrow?",
    "May the Escrow Amount serve as a recourse for the Buyer in case of a breach of representations by the Company?",
    "Are there any conditions to the closing?",
    "Are Change of Control Payments considered a Seller Transaction Expense?",
    "Would the aggregate amount payable by the Buyer to the Sellers be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount?",
    "Does the Buyer need to pay the Employees Closing Bonus Amount directly to the Company’s employees?",
    "Does any of the Sellers provide a representation with respect to any Tax matters related to the Company?",
    "Is any of the Sellers bound by a non-competition covenant after the Closing?",
    "Whose consent is required for the assignment of the Agreement by the Buyer?",
    "Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?"
]


context = [
 "In the agreement, it is specified that except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section 10.01). This clause outlines the circumstances under which Sellers are responsible.",


    "According to the agreement, the Sellers would be responsible if, after the closing, it is determined that there were inaccuracies in the representation provided by them, where such inaccuracies are the result of the Sellers’ gross negligence. This is outlined in detail in the agreement, section [specific section].",


    "The escrow amount specified in the agreement is $1,000,000. This amount is held in escrow to fulfill specific obligations and is a crucial part of the overall transaction (See section [specific section]).",


    "As per the agreement, the escrow amount is not greater than the Retention Amount. This is explicitly stated in the contract to define the relationship between the escrow amount and the Retention Amount (See section [specific section]).",


    "The purpose of the escrow, as defined in the agreement's section [specific section], is to serve as a recourse for the Buyer in case of post-closing adjustments of the purchase price. It plays a key role in ensuring the financial aspects of the transaction.",


    "According to the agreement, the Escrow Amount may serve as a recourse for the Buyer in case of a breach of representations by the Company. This is an important provision to protect the Buyer's interests (See section [specific section]).",


    "The agreement outlines specific conditions to the closing. Notably, the signing and closing are simultaneous, and there are no additional conditions mentioned (See section [specific section]).",


    "Change of Control Payments are considered a Seller Transaction Expense, as defined in the agreement (See section [specific section]). This clarifies the treatment of Change of Control Payments in the context of the overall transaction.",


    "The aggregate amount payable by the Buyer to the Sellers would be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount. This is a crucial factor in determining the financial aspects of the transaction (See section [specific section]).",


    "As per the agreement's Section 2.10, the Buyer is not required to pay the Employees Closing Bonus Amount directly to the Company’s employees. The specific details of payment are outlined in the agreement.",


    "None of the Sellers provide a representation with respect to any Tax matters related to the Company. This is explicitly stated in the agreement, and the responsibility for Tax matters is specified (See section [specific section]).",


    "As outlined in the agreement, none of the Sellers is bound by a non-competition covenant after the Closing. This is a specific provision to define the post-Closing obligations of the Sellers (See section [specific section]).",


    "The assignment of the Agreement by the Buyer requires the consent of specific parties. If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets, no consent is required. Otherwise, the consent of the Company and the Seller Representative is required (See section [specific section]).",


    "The Buyer needs the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate. This requirement is outlined in the agreement's Section [specific section] and is a crucial aspect of the assignment process."
]

ground_truth = [
    "Except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section  10.01)",
    "Yes",
    "$1,000,000",
    "No",
    "To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section  2.07(e))",
    "No",
    "No, as the signing and closing are simultaneous.",
    "Yes. (See defining Sellers Transaction Expenses).",
    "Yes (See Section  2.07)",
    "No. (See Section  2.10)",
    "No. Only the Company provides such a representation.",
    "No.",
    "If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets, no consent is required. Otherwise, the consent of the Company and the Seller Representative is required.",
    "No. If the assignment is not part of a sale of all or substantially all of the Buyer’s assets, the assignment requires the consent of the Company and the Seller’s Representative."
]

print(type(questions))



<class 'list'>


In [18]:
question_list = [q for q in questions]

print(question_list)


['Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?', 'Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?', 'How much is the escrow amount?', 'Is the escrow amount greater than the Retention Amount?', 'What is the purpose of the escrow?', 'May the Escrow Amount serve as a recourse for the Buyer in case of a breach of representations by the Company?', 'Are there any conditions to the closing?', 'Are Change of Control Payments considered a Seller Transaction Expense?', 'Would the aggregate amount payable by the Buyer to the Sellers be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount?', 'Does the Buyer need to pay the Employees Closing Bonus Amount directly to the Company’s employees?', 'Does any 

In [19]:
context_list = [[a] for a in context]
print(context_list)

[['In the agreement, it is specified that except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section 10.01). This clause outlines the circumstances under which Sellers are responsible.'], ['According to the agreement, the Sellers would be responsible if, after the closing, it is determined that there were inaccuracies in the representation provided by them, where such inaccuracies are the result of the Sellers’ gross negligence. This is outlined in detail in the agreement, section [specific section].'], ['The escrow amount specified in the agreement is $1,000,000. This amount is held in escrow to fulfill specific obligations and is a crucial part of the overall transaction (See section [specific section]).'], ['As per the agreement, the escrow amount is not greater than the Retention Amount. This is explicitly stated in the contract to define the relationship between the escrow amount and the Retention Amount (See section [spe

In [20]:
ground_truth_list = [[g] for g in ground_truth]
print(ground_truth_list)

[['Except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section  10.01)'], ['Yes'], ['$1,000,000'], ['No'], ['To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section  2.07(e))'], ['No'], ['No, as the signing and closing are simultaneous.'], ['Yes. (See defining Sellers Transaction Expenses).'], ['Yes (See Section  2.07)'], ['No. (See Section  2.10)'], ['No. Only the Company provides such a representation.'], ['No.'], ['If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets, no consent is required. Otherwise, the consent of the Company and the Seller Representative is required.'], ['No. If the assignment is not part of a sale of all or substantially all of the Buyer’s assets, the assignment requires the consent of the Company and the Seller’s Representative.']]


In [68]:
answers = []  
for question in questions:
    processed_question = str(question.lower().strip())
    print(processed_question)
    answerQ =chain({"question": processed_question})
    print(answerQ)
    answerA = answerQ["answer"]
    answers.append(answerA)
answers  

under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?
{'question': 'under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?', 'chat_history': [HumanMessage(content='Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?'), AIMessage(content='The text provided does not include information on whether the Buyer needs the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate.'), HumanMessage(content='Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?'), AIMessage(content="Apologies, but I can't answer your question as no specific question was provided. Could you please provide more details or clarify your question?"), HumanMessage(content='

["The content is a contractual agreement outlining the conditions for transaction between a buyer and seller of certain companies. \n\nSection II.23 states that the seller has provided all necessary information to the buyer, ensuring there are no misleading facts. \n\nArticle III outlines the sellers' responsibilities, stating that each seller is legally authorized to sell and that the transaction does not violate any laws or contractual obligations. \n\nArticle IV pertains to the buyer, asserting that they are also legally authorized to buy and the transaction does not violate any of their obligations.\n\nSection III.06 asserts that sellers have no liability to any broker, finder, or agent related to the transaction.\n\nSections I.04 and III.05 state that any public announcements about the transaction must be agreed on by all parties and that the sellers have the right to sell the equity interests they claim to hold, respectively.",
 "This document delineates the responsibilities and 

In [30]:
answer_list = [a for a in answers]
print(answer_list)

["The text sets out the terms and conditions for a stock purchase agreement. The agreement outlines the authority of the sellers' representative to act on behalf of the sellers, including executing documents, making decisions, and receiving notices related to the agreement. The sellers' representative is not liable for actions or omissions unless they involve gross negligence, bad faith, or willful misconduct. The agreement also includes sections on full disclosure and the authority, organization, and obligations of the sellers. The sellers have the right to retain legal counsel from Naschitz, Brandes, Amir & Co. for any disputes arising from the agreement. The agreement can be executed in any number of counterparts and becomes effective when signed and delivered by each party.", "The Sellers' Representative is authorized by the Sellers to perform various tasks and actions in connection with the closing of a transaction. These tasks include executing documents, making decisions, resolv

In [31]:
from datasets import Dataset
data = {
    "question": question_list, # list 
    "answer": answer_list, # list
    "contexts": context_list, # list list
    "ground_truths": ground_truth_list # list Lists
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 14
})


# RAGAS EVALUATION FOR THE SIMPLE RAG 

### SIMPLE RAG USING CONVERSATIONAL MEMORY

In [85]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        
    ],
)

df = result.to_pandas()

evaluating with [context_precision]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.06s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|██████████| 1/1 [00:04<00:00,  4.39s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|██████████| 1/1 [00:38<00:00, 38.20s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:13<00:00, 13.11s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [86]:
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,Under what circumstances and to what extent th...,The provided text does not contain specific in...,"[In the agreement, it is specified that except...","[Except in the case of fraud, the Sellers have...",1.0,1.0,0.000000,0.000000
1,Would the Sellers be responsible if after the ...,The provided context does not specifically add...,"[According to the agreement, the Sellers would...",[Yes],1.0,1.0,0.000000,0.000000
2,How much is the escrow amount?,"The Escrow Amount is $1,000,000.",[The escrow amount specified in the agreement ...,"[$1,000,000]",1.0,1.0,1.000000,0.939582
3,Is the escrow amount greater than the Retentio...,The text does not provide information on wheth...,"[As per the agreement, the escrow amount is no...",[No],1.0,1.0,0.000000,0.000000
4,What is the purpose of the escrow?,"The purpose of the escrow, as described in the...","[The purpose of the escrow, as defined in the ...",[To serve as a recourse of the Buyer in case o...,1.0,1.0,0.285714,0.962573
5,May the Escrow Amount serve as a recourse for ...,The text specifies that recovery from the Escr...,"[According to the agreement, the Escrow Amount...",[No],1.0,0.0,0.500000,0.000000
6,Are there any conditions to the closing?,The document provided does not specify the con...,[The agreement outlines specific conditions to...,"[No, as the signing and closing are simultaneo...",0.0,1.0,0.000000,0.000000
7,Are Change of Control Payments considered a Se...,The provided document does not explicitly stat...,[Change of Control Payments are considered a S...,[Yes. (See defining Sellers Transaction Expens...,1.0,0.5,0.000000,0.967186
8,Would the aggregate amount payable by the Buye...,"Yes, the aggregate amount payable by the buyer...",[The aggregate amount payable by the Buyer to ...,[Yes (See Section 2.07)],1.0,1.0,1.000000,0.990314
9,Does the Buyer need to pay the Employees Closi...,"No, it is not necessary for the buyer to direc...","[As per the agreement's Section 2.10, the Buye...",[No. (See Section 2.10)],1.0,1.0,0.333333,0.970163


### RAGAS EVALUATION FOR ENTITY MEMORY

In [2]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [7]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm,),

)

In [8]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'That sounds exciting! Hackathons are a great way to challenge yourself and create innovative solutions. Can you tell me more about the project Deven and Sam are working on? For instance, what is the theme or the problem they are trying to solve? The more details you provide, the better I can assist with any questions or challenges they might be facing.'

In [70]:
from datasets import Dataset
data = {
    "question": question_list, # list 
    "answer": answer_list, # list
    "contexts": context_list, # list list
    "ground_truths": ground_truth_list # list Lists
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 14
})


In [71]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        
    ],
)

df = result.to_pandas()

evaluating with [context_precision]


100%|██████████| 1/1 [00:05<00:00,  5.02s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|██████████| 1/1 [00:07<00:00,  7.96s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|██████████| 1/1 [00:43<00:00, 43.98s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:13<00:00, 13.89s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [72]:
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,Under what circumstances and to what extent th...,The content is a contractual agreement outlini...,"[In the agreement, it is specified that except...","[Except in the case of fraud, the Sellers have...",1.0,1.0,0.333333,0.788861
1,Would the Sellers be responsible if after the ...,This document delineates the responsibilities ...,"[According to the agreement, the Sellers would...",[Yes],1.0,1.0,0.000000,0.818457
2,How much is the escrow amount?,The parties have agreed to cooperate with an A...,[The escrow amount specified in the agreement ...,"[$1,000,000]",1.0,1.0,0.125000,0.904549
3,Is the escrow amount greater than the Retentio...,The parties will cooperate with the Accounting...,"[As per the agreement, the escrow amount is no...",[No],1.0,1.0,1.000000,0.922271
4,What is the purpose of the escrow?,The provided content is a legal document discu...,"[The purpose of the escrow, as defined in the ...",[To serve as a recourse of the Buyer in case o...,1.0,1.0,0.000000,0.982255
5,May the Escrow Amount serve as a recourse for ...,This agreement outlines the terms of a busines...,"[According to the agreement, the Escrow Amount...",[No],1.0,0.0,0.222222,0.798366
6,Are there any conditions to the closing?,The parties agreed on the final closing statem...,[The agreement outlines specific conditions to...,"[No, as the signing and closing are simultaneo...",0.0,1.0,0.375000,0.913074
7,Are Change of Control Payments considered a Se...,The document sections provide details on the a...,[Change of Control Payments are considered a S...,[Yes. (See defining Sellers Transaction Expens...,1.0,0.5,0.833333,0.757605
8,Would the aggregate amount payable by the Buye...,The content refers to the purchase price adjus...,[The aggregate amount payable by the Buyer to ...,[Yes (See Section 2.07)],1.0,1.0,0.666667,0.810876
9,Does the Buyer need to pay the Employees Closi...,This document refers to an agreement concernin...,"[As per the agreement's Section 2.10, the Buye...",[No. (See Section 2.10)],1.0,1.0,0.000000,0.775223


### RAGAS EVALUATION FOR SMALLER ANSWER

In [82]:
answers = []  
for question in questions:
    processed_question = str(question.lower().strip())
    print(processed_question)
    answerQ =chain({"question": processed_question})
    print(answerQ)
    answerA = answerQ["answer"]
    answers.append(answerA)
answers  

under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?
{'question': 'under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?', 'chat_history': [HumanMessage(content='Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?'), AIMessage(content='The text provided does not include information on whether the Buyer needs the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate.'), HumanMessage(content='Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?'), AIMessage(content="Apologies, but I can't answer your question as no specific question was provided. Could you please provide more details or clarify your question?"), HumanMessage(content='

["The document outlines the responsibilities and powers of a Sellers' Representative in connection with a stock purchase agreement. The Sellers’ Representative is authorized to execute and receive all necessary documents, receive and give notices, make all filings, make decisions, and take actions on behalf of the Sellers. They are also authorized to manage the escrow account associated with the transaction. The Sellers’ Representative is not liable for actions or omissions unless they are grossly negligent, act in bad faith, or engage in willful misconduct. The Sellers are required to indemnify the Sellers’ Representative for any losses arising from their role, excluding those caused by the aforementioned exceptions.\n\nIn addition, the document also states that the Sellers are organized and authorized to execute this agreement and that the execution of this agreement does not violate any legal requirement or cause a breach or violation of any other agreement.\n\nThe document also spe

In [32]:
from datasets import Dataset
data = {
    "question": question_list, # list 
    "answer": answer_list, # list
    "contexts": context_list, # list list
    "ground_truths": ground_truth_list # list Lists
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 14
})


In [85]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        
    ],
)

df = result.to_pandas()

evaluating with [context_precision]


100%|██████████| 1/1 [00:06<00:00,  6.15s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|██████████| 1/1 [00:07<00:00,  7.23s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|██████████| 1/1 [00:47<00:00, 47.12s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:13<00:00, 13.86s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [39]:
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,Under what circumstances and to what extent th...,The text sets out the terms and conditions for...,"[In the agreement, it is specified that except...","[Except in the case of fraud, the Sellers have...",1.0,1.0,0.142857,0.987786
1,Would the Sellers be responsible if after the ...,The Sellers' Representative is authorized by t...,"[According to the agreement, the Sellers would...",[Yes],1.0,1.0,0.666667,0.797862
2,How much is the escrow amount?,The document outlines the regulations and proc...,[The escrow amount specified in the agreement ...,"[$1,000,000]",1.0,1.0,0.166667,0.904549
3,Is the escrow amount greater than the Retentio...,The parties in this agreement are engaging an ...,"[As per the agreement, the escrow amount is no...",[No],1.0,1.0,0.000000,0.935104
4,What is the purpose of the escrow?,The parties have agreed to a Final Closing Sta...,"[The purpose of the escrow, as defined in the ...",[To serve as a recourse of the Buyer in case o...,1.0,1.0,1.000000,0.982255
5,May the Escrow Amount serve as a recourse for ...,The parties have agreed to a final closing sta...,"[According to the agreement, the Escrow Amount...",[No],1.0,0.0,0.000000,0.960084
6,Are there any conditions to the closing?,The content is an excerpt from a legal agreeme...,[The agreement outlines specific conditions to...,"[No, as the signing and closing are simultaneo...",0.0,1.0,0.571429,0.802166
7,Are Change of Control Payments considered a Se...,The document appears to be a legal agreement b...,[Change of Control Payments are considered a S...,[Yes. (See defining Sellers Transaction Expens...,1.0,0.5,0.125000,0.742687
8,Would the aggregate amount payable by the Buye...,The document outlines the process of purchase ...,[The aggregate amount payable by the Buyer to ...,[Yes (See Section 2.07)],1.0,1.0,0.571429,0.741814
9,Does the Buyer need to pay the Employees Closi...,The content discusses tax withholding in accor...,"[As per the agreement's Section 2.10, the Buye...",[No. (See Section 2.10)],1.0,1.0,0.000000,0.893559


### SIMPLE PROMPT MODIFICATION

In [21]:
answers = []  
for question in questions:
    processed_question = str(question.lower().strip())
    print(processed_question)
    answerQ =chain({"question": processed_question})
    print(answerQ)
    answerA = answerQ["answer"]
    answers.append(answerA)
answers  

under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?
{'question': 'under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?', 'chat_history': [HumanMessage(content='Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?'), AIMessage(content="The document discusses the terms of a transaction involving the sale of shares, options, or warrants from a seller to a buyer. The seller is the legal owner and has the authority to sell their equity interests, which are free of any encumbrances. After the sale, the buyer will become the legal owner of the equity interests. The transaction does not require any action or authorization from any third party, and will not create any encumbrances. The execution and delivery of this agreement are within the buyer's power and authority. The partie

["This document appears to be a section from a legal agreement, likely related to a business transaction or merger. Specific details include:\n\n- Provisions outlining that neither party can use any item or matter specified in the agreement or any schedule in any dispute or controversy regarding whether these items or matters are in the ordinary course of business or align with past practice.\n- In any dispute arising from this Agreement, the Sellers have the right to retain the legal firm Naschitz, Brandes, Amir & Co. (NBA) to represent them, even if this representation is against the Buyer or the Acquired Companies.\n- The Buyer, for itself and the Acquired Companies, consents to any such representation and waives any actual or potential conflict arising from such representation.\n- All communications between or among NBA, the Company, and the Company Securityholders related to the Agreement and the Ancillary Agreements will be deemed to be attorney-client privileged.\n- The document

In [26]:
answers = []  
for question in questions:
    processed_question = str(question.lower().strip())
    print(processed_question)
    answerQ =chain({"question": processed_question})
    print(answerQ)
    answerA = answerQ["answer"]
    answers.append(answerA)
answers  

under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?
{'question': 'under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?', 'chat_history': [HumanMessage(content='under what circumstances and to what extent the sellers are responsible for a breach of representations and warranties?'), AIMessage(content="The text sets out the terms and conditions for a stock purchase agreement. The agreement outlines the authority of the sellers' representative to act on behalf of the sellers, including executing documents, making decisions, and receiving notices related to the agreement. The sellers' representative is not liable for actions or omissions unless they involve gross negligence, bad faith, or willful misconduct. The agreement also includes sections on full disclosure and the authority, organization, and obligations of the sellers. The sellers have the right to 

["The text sets out the terms and conditions for a stock purchase agreement. The agreement outlines the authority of the sellers' representative to act on behalf of the sellers, including executing documents, making decisions, and receiving notices related to the agreement. The sellers' representative is not liable for actions or omissions unless they involve gross negligence, bad faith, or willful misconduct. The agreement also includes sections on full disclosure and the authority, organization, and obligations of the sellers. The sellers have the right to retain legal counsel from Naschitz, Brandes, Amir & Co. for any disputes arising from the agreement. The agreement can be executed in any number of counterparts and becomes effective when signed and delivered by each party.",
 "The Sellers' Representative is authorized by the Sellers to perform various tasks and actions in connection with the closing of a transaction. These tasks include executing documents, making decisions, resol

In [33]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        
    ],
)

df = result.to_pandas()

evaluating with [context_precision]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:06<00:00,  6.09s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|██████████| 1/1 [00:07<00:00,  7.63s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|██████████| 1/1 [00:50<00:00, 50.46s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:13<00:00, 13.66s/it]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/lillian/.local/lib/python3.10/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [34]:
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,Under what circumstances and to what extent th...,The text sets out the terms and conditions for...,"[In the agreement, it is specified that except...","[Except in the case of fraud, the Sellers have...",1.0,1.0,0.142857,0.987786
1,Would the Sellers be responsible if after the ...,The Sellers' Representative is authorized by t...,"[According to the agreement, the Sellers would...",[Yes],1.0,1.0,0.666667,0.797862
2,How much is the escrow amount?,The document outlines the regulations and proc...,[The escrow amount specified in the agreement ...,"[$1,000,000]",1.0,1.0,0.166667,0.904549
3,Is the escrow amount greater than the Retentio...,The parties in this agreement are engaging an ...,"[As per the agreement, the escrow amount is no...",[No],1.0,1.0,0.000000,0.935104
4,What is the purpose of the escrow?,The parties have agreed to a Final Closing Sta...,"[The purpose of the escrow, as defined in the ...",[To serve as a recourse of the Buyer in case o...,1.0,1.0,1.000000,0.982255
5,May the Escrow Amount serve as a recourse for ...,The parties have agreed to a final closing sta...,"[According to the agreement, the Escrow Amount...",[No],1.0,0.0,0.000000,0.960084
6,Are there any conditions to the closing?,The content is an excerpt from a legal agreeme...,[The agreement outlines specific conditions to...,"[No, as the signing and closing are simultaneo...",0.0,1.0,0.571429,0.802166
7,Are Change of Control Payments considered a Se...,The document appears to be a legal agreement b...,[Change of Control Payments are considered a S...,[Yes. (See defining Sellers Transaction Expens...,1.0,0.5,0.125000,0.742687
8,Would the aggregate amount payable by the Buye...,The document outlines the process of purchase ...,[The aggregate amount payable by the Buyer to ...,[Yes (See Section 2.07)],1.0,1.0,0.571429,0.741814
9,Does the Buyer need to pay the Employees Closi...,The content discusses tax withholding in accor...,"[As per the agreement's Section 2.10, the Buye...",[No. (See Section 2.10)],1.0,1.0,0.000000,0.893559
